In [1]:
import sys
import os

# Insert the parent directory of "app" into sys.path
# so that Python recognizes "app" as an importable package.
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(parent_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv()  # This looks for .env in the current working directory

True

In [3]:
import time

import pandas as pd
import seaborn as sns
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
)

In [4]:
# Benchmark data
test_df = pd.read_csv('benchmark.csv', delimiter=";")
test_df.head()

,Question,Answer
0,What are some contracts that ended after 1990-...,There are a total of 377 contracts that ended ...
1,What are some contracts that ended after 1990-...,"There are 48 contracts classified as ""Strategi..."
2,"What are some contracts with HC2 Holdings, Inc.?","There is one contract involving HC2 Holdings, ..."
3,Which year was the most contracts signed?,The year with the most contracts signed was 20...
4,Which company has the most active contracts?,The company with the most active contracts is ...


In [5]:
from backend.src.agent import get_agent

In [6]:
agent = get_agent()

In [7]:
from langchain_core.messages import HumanMessage

input_messages = [HumanMessage(content="Which year was the most contracts signed?")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Which year was the most contracts signed?
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (d3bf3576-c229-4029-90ea-a34c3e1a6fe3)
 Call ID: d3bf3576-c229-4029-90ea-a34c3e1a6fe3
  Args:
    cypher_aggregation: RETURN effective_date.year AS year, count(*) AS count ORDER BY year
================================= Tool Message =================================
Name: ContractSearch

[{"year": 1900, "count": 2}, {"year": 1990, "count": 1}, {"year": 1992, "count": 1}, {"year": 1993, "count": 1}, {"year": 1994, "count": 1}, {"year": 1995, "count": 2}, {"year": 1996, "count": 12}, {"year": 1997, "count": 10}, {"year": 1998, "count": 20}, {"year": 1999, "count": 26}, {"year": 2000, "count": 24}, {"year": 2001, "count": 8}, {"year": 2002, "count": 8}, {"year": 2003, "count": 11}, {"year": 2004, "count": 14}, {"year": 2005, "count": 20}, {"year": 2006, "coun

In [8]:
# History
messages

{'messages': [HumanMessage(content='Which year was the most contracts signed?', additional_kwargs={}, response_metadata={}, id='960fdae6-09b1-40e1-89cf-f28f584dd9e2'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'ContractSearch', 'arguments': '{"cypher_aggregation": "RETURN effective_date.year AS year, count(*) AS count ORDER BY year"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e4b81e4c-f5e9-4c1e-84ba-123298133a2a-0', tool_calls=[{'name': 'ContractSearch', 'args': {'cypher_aggregation': 'RETURN effective_date.year AS year, count(*) AS count ORDER BY year'}, 'id': 'd3bf3576-c229-4029-90ea-a34c3e1a6fe3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 532, 'output_tokens': 22, 'total_tokens': 554, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"year": 1900, "count": 2}, {"year": 1990, "count": 1}, {"year": 1992, "count": 1}, {"year": 1993

In [9]:
# Chat with history

input_messages = messages['messages'] + [HumanMessage(content="Which party has the most active contracts?")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Which year was the most contracts signed?
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (d3bf3576-c229-4029-90ea-a34c3e1a6fe3)
 Call ID: d3bf3576-c229-4029-90ea-a34c3e1a6fe3
  Args:
    cypher_aggregation: RETURN effective_date.year AS year, count(*) AS count ORDER BY year
================================= Tool Message =================================
Name: ContractSearch

[{"year": 1900, "count": 2}, {"year": 1990, "count": 1}, {"year": 1992, "count": 1}, {"year": 1993, "count": 1}, {"year": 1994, "count": 1}, {"year": 1995, "count": 2}, {"year": 1996, "count": 12}, {"year": 1997, "count": 10}, {"year": 1998, "count": 20}, {"year": 1999, "count": 26}, {"year": 2000, "count": 24}, {"year": 2001, "count": 8}, {"year": 2002, "count": 8}, {"year": 2003, "count": 11}, {"year": 2004, "count": 14}, {"year": 2005, "count": 20}, {"year": 2006, "coun